# Runs brainvisa preprocessing

This notebook creates the whole brainvisa preprocessing pipeline to feed to deep learning algorithms.

# Imports

General imports

In [ ]:
import sys
import os
from os.path import join

Deep_folding imports

In [ ]:
from deep_folding.anatomist_tools import transform
from deep_folding.anatomist_tools import bounding_box
from deep_folding.anatomist_tools import dataset_gen_pipe
print(os.path.dirname(transform.__file__))
print(os.path.dirname(bounding_box.__file__))
print(os.path.dirname(dataset_gen_pipe.__file__))

# Creates useful functions

In [ ]:
def check_directory(directory_path):
    """Checks directory path and returns absolute path"""
    directory_path = os.path.abspath(directory_path)
    if os.path.isdir(directory_path):
        print(directory_path + ' is a directory')
    else:
        print(directory_path + ' does not exist or is not a directory')
    return directory_path

# User-specific variables

We now assign path names and other user-specific variables.

In [ ]:
path_to_neurospin = '/neurospin'  # Could be either /neuropsin, or /nfs/neurospin, for example

The source directory is where the unsupervised database lies. It contains the morphologist analysis subfolder ANALYSIS/3T_morphologist


In [ ]:
unsupervised_src_dir = check_directory(join(path_to_neurospin, 'hcp'))

The transform directory is where the transform files will be saved. It will be created afterwards if it does not exist

In [ ]:
transform_dir = check_directory(join(path_to_neurospin, 'dico/deep_folding_data/data', 'transform'))

# Generates transform files

In [36]:
transform.transform_to_spm(src_dir=unsupervised_src_dir, tgt_dir=transform_dir,
                           number_subjects=transform._ALL_SUBJECTS)

subject : 129533
subject : 139839
subject : 192439
subject : 552544


KeyboardInterrupt: 

# Result analysis

Prints the list of files of the target directory

In [ ]:
print('\n'.join(os.listdir(tgt_dir)))

Expected output (we read the transformation file from the reference directory):

In [ ]:
ref_file = os.listdir(ref_dir)[0]
print "ref_file = ", ref_file, '\n'
with open(os.path.join(ref_dir,ref_file), 'r') as f:
    print(f.read())

Obtained output (we read the transformation file from the target directory):

In [ ]:
tgt_file = os.listdir(tgt_dir)[0]
print"tgt_file = ", tgt_file, '\n'
with open(os.path.join(tgt_dir,tgt_file), 'r') as f:
    print(f.read())

Generated README (we read the generated README from the target directory)

In [ ]:
with open(os.path.join(tgt_dir,"transform.json"), 'r') as f:
    print(f.read())
    print type(f.read())

In [ ]:
from soma import aims

In [ ]:
t = aims.read(os.path.join(tgt_dir,tgt_file))

In [ ]:
t

In [ ]:
r = aims.read(os.path.join(ref_dir,ref_file))

In [ ]:
r

In [ ]:
t == r